#Clustering the Findings

In [0]:
#Libraries
import pandas as pd
import re
import nltk

from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.cluster import KMeans

#Paths
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
DF_PARSED = DATASETS_FOLDER + 'DF_ReportsInColumns_v19.11.29.pkl'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Import FILES
#Reports already Parsed and in columns
dfRepO=pd.read_pickle(DF_PARSED)

In [0]:
dfRepO

,OrgSubCod,OrgSub,FileNumber,ReportNumber,ReportType,DateBegin,DateEnd,ConstNumber,ConstTYPE,ConstTitle,ConstText
0,,companhia de eletricidade do acre,13568,201900295,Auditoria Anual de Contas,2018/1,2018/12,1.,,a unidade nao apresentou a totalidade da docum...,a unidade nao apresentou a totalidade da docum...
2,,companhia de eletricidade do acre,13568,201900295,Auditoria Anual de Contas,2018/1,2018/12,3.,,,os autos iniciais dos processos de contas sera...
3,,empresa brasileira de hemoderivados e biotecno...,13552,201900153,Auditoria Anual de Contas,2018/1,2018/12,1.,,analise das demonstracoes contabeis. 13,analise das demonstracoes contabeis. no tocant...
4,,empresa brasileira de hemoderivados e biotecno...,13552,201900153,Auditoria Anual de Contas,2018/1,2018/12,2.,,divergencias entre os dados das demonstracoes ...,divergencias entre os dados das demonstracoes ...
5,,empresa brasileira de hemoderivados e biotecno...,13552,201900153,Auditoria Anual de Contas,2018/1,2018/12,3.,,resultados da gestao. 24,resultados da gestao. no tocante ao item 2 do ...
...,...,...,...,...,...,...,...,...,...,...,...
12555,,secretaria de articulacao institucional/minc,7123,243949,Avaliação dos Resultados da Gestão,2438/1,2438/12,1.350,,,eleitos nas conferencias estaduais e ate 135 d...
12556,,secretaria de articulacao institucional/minc,7123,243949,Avaliação dos Resultados da Gestão,2438/1,2438/12,1.800,,,"representantes em brasilia, provenientes de to..."
12557,,secretaria de articulacao institucional/minc,7123,243949,Avaliação dos Resultados da Gestão,2438/1,2438/12,2010.,,,i escopo do trabalho os trabalhos foram realiz...
12558,,secretaria de articulacao institucional/minc,7123,243949,Avaliação dos Resultados da Gestão,2438/1,2438/12,10.,,,i escopo do trabalho os trabalhos foram realiz...


In [0]:
#dfRepO[dfRepO['FileNumber']=='13552']['ConstText']

In [0]:
#Some Stats of dfRep
# dfRep.head()
print( pd.pivot_table( dfRepO, values=['ConstNumber'],                          
                          columns=['ConstTYPE'],
                          aggfunc=len ))

ConstTYPE           conclusao  constatacao  informacao
ConstNumber  39987       5348        23264       20500


In [0]:
#TEST to RELATE with

In [0]:
#Select just Rows with Conclusao,Constatacao and Informacao
dfRep = dfRepO[ ~(dfRepO.ConstTYPE=='') ]
print(len(dfRep))

#Small Select for TEST
dfRep = dfRep[:500]
print(len(dfRep))


49112
500


In [0]:
#NORMALIZE_TOKEN
stop_words = nltk.corpus.stopwords.words('portuguese')
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer('portuguese')

def removeUnWantedWords(text):

  # Remove numbers/dates/pages 
  cleanText = re.sub('\d', " ", text) #Remove all Numbers
  # cleanText = re.sub('\s\d+|\d+\s', " ", text)

  # Remove months
  month = "(janeiro|fevereiro|marco|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)"
  cleanText = re.sub('\s'+month+'\s', " ", cleanText)
  return cleanText.strip()

def normalize_token_corpus(corpus, 
                           stem=True,
                           stop_words_IF=True,
                           tokenize=True):
    
    normalized_token_corpus = []    
    for text in corpus:
        
        # Remove Unwanted Numbers/Dates/NumberPages
        text=removeUnWantedWords(text)
        
        # Tokenize the input string
        tokens = tokenizer.tokenize(text.lower())

        # Remove the stop words 
        if stop_words_IF:    
          tokens = [x for x in tokens if not x in stop_words]
    
        # Perform stemming on the tokenized words 
        if stem:
          tokens = [stemmer.stem(x) for x in tokens]

        if tokenize:
            normalized_token_corpus.append(tokens)
        else:
            tokens=" ".join(tokens)
            normalized_token_corpus.append(tokens)
            
    return normalized_token_corpus

In [0]:
#TF_IDF
def tf_idf(corpus,
           ngram_range=(1, 1),
           min_df=0.0,
           max_df=1.0):
  
  vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range)
  feature_matrix = vectorizer.fit_transform(corpus).astype(float)

  return vectorizer, feature_matrix

#TESTE:
# feature_matrix.shape
# feature_names = vectorizer.get_feature_names()
# print(feature_names[:100])

In [0]:
#K_means Cluster
def k_means(feature_matrix, num_clusters=5):
    km = KMeans(n_clusters=num_clusters,
                max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters


In [0]:
########## MAIN
# Corpus
corpus=dfRep.ConstText.to_list()
print(corpus[1:10])
corpus_token=normalize_token_corpus(corpus,tokenize=False)

# TF_IDF
vectorizer, feature_matrix = tf_idf(corpus_token,
                                    min_df=0.24, 
                                    max_df=0.85,
                                    ngram_range=(1, 2))

# KMeans   
num_clusters = 3 #(Information, Conclusion, Findings) 

km_obj, clusters = k_means(feature_matrix=feature_matrix,
                            num_clusters=num_clusters)

dfResult = dfRep[['ReportNumber', 'ConstTYPE' ] ]
dfResult['Cluster']= clusters

[' foi efetuada analise com o objetivo de certificar que a gratificacao de desempenho de atividade geocientificas gdag somente foi concedida aos funcionarios com provimento efetivo. conforme previsto no plano de carreiras, cargos e salarios pccs da cprm, mais especificamente no item 6.2.1, a gdag e devida aos empregados de cargos de provimento efetivo da empresa, quando em exercicio das suas atividades. portanto, a concessao da gdag e destinada exclusivamente aos empregados efetivos da companhia, nao havendo previsao normativa para que a citada gratificacao seja concedida a outras formas de contratacao. por meio da analise dos contracheques constatou-se que a companhia estendeu indevidamente o pagamento da gdag a 62 contratados ocupantes de cargos em comissao, o que contraria o item 6.2.1 do plano de carreiras, cargos e salarios pccs da cprm, conforme destacado em item especifico deste relatorio. o uso indevido da gdag proporcionou, no exercicio em exame, pagamentos no valor total de r

500

In [0]:
# Test Result
dfResult.pivot_table(values='ReportNumber', index='ConstTYPE', columns='Cluster', aggfunc='count' )

Cluster,0,1,2
ConstTYPE,,,
conclusao,19.0,NaN,22.0
constatacao,26.0,67.0,204.0
informacao,5.0,26.0,131.0


In [0]:
#Write DF to disk
# fileDFCluster='/content/drive/My Drive/Research/DataSets/DF_ReportsInColumns_AAG_CLUSTER_v19.10.25.pkl'
# dfRepTest.to_pickle(fileDFCluster)


In [0]:
# GRAPH for VISUALIZATION
# (Example of the book)